# 🤖 Análise de Consenso entre LLMs - Anotação sem LLM Hacking

## 1) Setup e Configuração

In [70]:
import sys
from loguru import logger
import pandas as pd

logger.remove()
logger.add(
    sys.stdout,
    format="<green>{time:HH:mm:ss}</green> | <level>{level: <8}</level> | <level>{message}</level>",
    level="INFO"
)

logger.success("✓ Setup completo")

12:51:33 | SUCCESS  | ✓ Setup completo


## 2) Carregar Dataset

In [71]:
from src.utils.data_loader import load_hf_dataset, load_hf_dataset_as_dataframe, list_available_datasets, get_dataset_info

logger.info("Datasets disponíveis:")
for dataset in list_available_datasets():
    logger.info(f"  - {dataset}")

12:51:33 | INFO     | Datasets disponíveis:
12:51:33 | INFO     |   - agnews
12:51:33 | INFO     |   - mpqa
12:51:33 | INFO     |   - webkb
12:51:33 | INFO     |   - ohsumed
12:51:33 | INFO     |   - acm
12:51:33 | INFO     |   - yelp_2013
12:51:33 | INFO     |   - dblp
12:51:33 | INFO     |   - books
12:51:33 | INFO     |   - reut90
12:51:33 | INFO     |   - wos11967
12:51:33 | INFO     |   - twitter
12:51:33 | INFO     |   - trec
12:51:33 | INFO     |   - wos5736
12:51:33 | INFO     |   - sst1
12:51:33 | INFO     |   - pang_movie
12:51:33 | INFO     |   - movie_review
12:51:33 | INFO     |   - vader_movie
12:51:33 | INFO     |   - subj
12:51:33 | INFO     |   - sst2
12:51:33 | INFO     |   - yelp_reviews
12:51:33 | INFO     |   - 20ng
12:51:33 | INFO     |   - medline


In [93]:
dataset_name = "wos5736"  # Ajuste conforme necessário

texts, categories, ground_truth = load_hf_dataset(dataset_name)

logger.info(f"Textos: {len(texts)}")
logger.info(f"Categorias: {categories}")
logger.info(f"Ground truth: {'Sim' if ground_truth else 'Não'}")

12:52:14 | INFO     | Carregando dataset: wos5736
12:52:14 | INFO     | Combinando splits: ['train', 'test']
12:52:22 | INFO     |   ✓ train: 51624 exemplos
12:52:29 | INFO     |   ✓ test: 5736 exemplos
12:52:29 | INFO     | Total combinado: 57360 exemplos
12:52:29 | INFO     | Categorias extraídas automaticamente: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
12:52:29 | INFO     | Amostra reduzida para 500 exemplos (seed=42)
12:52:29 | INFO     | Coluna de texto: text
12:52:29 | INFO     | Ground truth carregado da coluna 'label'
12:52:29 | INFO     | Textos: 500
12:52:29 | INFO     | Categorias: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
12:52:29 | INFO     | Ground truth: Sim


In [94]:
# Visualizar amostra
logger.info("Amostra dos textos:")
for i, text in enumerate(texts[:3]):
    logger.info(f"{i+1}. {text[:100]}...")
    if ground_truth:
        logger.info(f"   Label: {ground_truth[i]}")

12:52:29 | INFO     | Amostra dos textos:
12:52:29 | INFO     | 1. A universal feature of the replication of positive-strand RNA viruses is the association with intrac...
12:52:29 | INFO     |    Label: 7
12:52:29 | INFO     | 2. This paper presents an ultra-low-power low-voltage Class-AB Fully Differential Operational Amplifier...
12:52:29 | INFO     |    Label: 2
12:52:29 | INFO     | 3. The human microbiome consists of all microorganisms occupying the skin, mucous membranes and intesti...
12:52:29 | INFO     |    Label: 8


In [95]:
df, categories = load_hf_dataset_as_dataframe(dataset_name)

12:52:29 | INFO     | Carregando dataset: wos5736
12:52:29 | INFO     | Combinando splits: ['train', 'test']


Using the latest cached version of the dataset since waashk/wos5736 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at ..\..\data\.cache\hf\waashk___wos5736\default\0.0.0\cb34f4d2b1bb4e986578ee69a2dbc0759d4b6b7c (last modified on Tue Dec  2 12:32:22 2025).


12:52:35 | INFO     |   ✓ train: 51624 exemplos


Using the latest cached version of the dataset since waashk/wos5736 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at ..\..\data\.cache\hf\waashk___wos5736\default\0.0.0\cb34f4d2b1bb4e986578ee69a2dbc0759d4b6b7c (last modified on Tue Dec  2 12:32:22 2025).


12:52:35 | INFO     |   ✓ test: 5736 exemplos
12:52:35 | INFO     | Total combinado: 57360 exemplos
12:52:35 | INFO     | Categorias extraídas automaticamente: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
12:52:35 | INFO     | Amostra reduzida para 500 exemplos (seed=42)
12:52:35 | INFO     | Coluna de texto: text
12:52:35 | INFO     | Ground truth carregado da coluna 'label'
12:52:35 | INFO     | DataFrame criado com 500 linhas


## 3) Configurar Modelos LLM

### Opção A: Usar apenas parâmetros padrão (temp=0)

In [96]:
from src.llm_annotation_system.annotation.llm_annotator import LLMAnnotator
from src.experiments.base_experiment import DEFAULT_MODELS, BASE_ANNOTATION_PROMPT

annotator = LLMAnnotator(
    dataset_name=dataset_name,
    categories=categories,
    models=DEFAULT_MODELS,
    prompt_template=BASE_ANNOTATION_PROMPT,
    use_langchain_cache=True,
    use_alternative_params=False  # Apenas temp=0
)

logger.success(f"✓ Annotator inicializado com {len(annotator.models)} modelos")

12:52:35 | INFO     | Cache carregado: 14710 entradas
12:52:35 | INFO     | Cache LangChain ativado: ..\..\data\.cache\langchain\langchain_cache.db
12:52:35 | INFO     | Template do prompt preparado
12:52:35 | INFO     | LLMAnnotator inicializado
12:52:35 | INFO     | Modelos: 5 | Categorias: 11
12:52:35 | SUCCESS  | ✓ Annotator inicializado com 5 modelos


## 4) Executar Anotação

In [97]:
df

,text,label,label_description
0,A universal feature of the replication of posi...,7,Molecular biology
1,This paper presents an ultra-low-power low-vol...,2,Operational amplifier
2,The human microbiome consists of all microorga...,8,Immunology
3,Background: Limited accessibility to providers...,8,Immunology
4,Chronic myeloid leukemia (CML) is a hematologi...,8,Immunology
...,...,...,...
495,Objective. -To determine if bacterial vaginosi...,7,Molecular biology
496,Substance use and depression are prevalent amo...,4,Child abuse
497,Aims: Understanding the metabolism of lactose ...,10,Northern blotting
498,Physical treatments have gained great interest...,7,Molecular biology


In [98]:
annotator.annotation_engine.template

'You are an expert data annotator with extensive experience in text classification tasks.\n\nYour task is to classify the following text into one of the predefined categories with high precision.\n\n**Instructions:**\n1. Read the text carefully and understand its context\n2. Consider the nuances and implicit meanings\n3. Select the most appropriate category based on the content\n4. Be consistent with your classification criteria\n5. If the text is ambiguous, choose the most likely category based on dominant features\n\n**Available Categories:**\n- 0: Electricity\n- 1: Digital control\n- 2: Operational amplifier\n- 3: Social cognition\n- 4: Child abuse\n- 5: Depression\n- 6: Attention\n- 7: Molecular biology\n- 8: Immunology\n- 9: Polymerase chain reaction\n- 10: Northern blotting\n\n**Text to classify (Scientific abstract):**\n{text}\n\n**Important Guidelines:**\n- Provide ONLY the category number as your response\n- Do not include explanations\n- Be objective and avoid bias\n- Conside

### - Testar anotação única

In [99]:
# Parâmetros de anotação
num_repetitions = 3
text = texts[2]
model = "llama3.1-8b"

total_annotation = len(text) * num_repetitions
logger.warning(f"  Modelo: {model}")
logger.warning(f"  Texto: {len(text)}")
logger.warning(f"  Repetições: {num_repetitions}")

annotations = annotator.annotate_single(
    text=text,
    model=model,
    num_repetitions=num_repetitions,
    use_cache=False
)

logger.success("✓ Anotação completa")

annotations

12:52:36 | WARNING  |   Modelo: llama3.1-8b
12:52:36 | WARNING  |   Texto: 1395
12:52:36 | WARNING  |   Repetições: 3
12:52:36 | SUCCESS  | ✓ Anotação completa


[7, 7, 7]

### - Anotando dataset completo

In [100]:
output_file = annotator.results_dir / "annotations_complete.csv"
use_old_annotations = False

if use_old_annotations and output_file.exists():
    logger.warning(f"Arquivo já existe: {output_file}")
    logger.warning("Carregando anotações salvas e pulando anotação...")
    df_annotations = pd.read_csv(output_file)
    logger.success("✓ Anotações carregadas do disco")
    
else: 
    num_repetitions = 1
    df_annotations = annotator.annotate_dataset(
        texts=texts,
        num_repetitions=num_repetitions,
        use_cache=True
    )


logger.success("✓ Anotações completas")
df_annotations

12:52:36 | INFO     | Iniciando anotação
12:52:36 | INFO     | Textos: 500 | Modelos: 5 | Repetições: 1
12:52:36 | INFO     | Total de anotações: 2500


Anotando:   0%|          | 0/500 [00:00<?, ?it/s]

12:52:36 | WARNING  | Falha ao converter resposta para inteiro e nenhum número encontrado: 'Immunology'
12:52:36 | WARNING  | Resposta é None ou ''


Anotando:  28%|██▊       | 140/500 [00:00<00:00, 1384.89it/s]

12:52:36 | WARNING  | Resposta é None ou ''
12:52:36 | WARNING  | Resposta é None ou ''


Anotando:  56%|█████▌    | 279/500 [00:00<00:00, 1118.50it/s]

12:52:36 | WARNING  | Resposta é None ou ''
12:52:36 | WARNING  | Resposta é None ou ''


Anotando: 100%|██████████| 500/500 [00:00<00:00, 1067.41it/s]

12:52:36 | SUCCESS  | Anotações completas salvas: ..\..\results\wos5736\annotations_complete.csv
12:52:36 | SUCCESS  | ✓ Anotações completas


,text_id,text,deepseek-r1-8b_rep1,deepseek-r1-8b_consensus,deepseek-r1-8b_consensus_score,qwen3-8b_rep1,qwen3-8b_consensus,qwen3-8b_consensus_score,gemma3-4b_rep1,gemma3-4b_consensus,gemma3-4b_consensus_score,mistral-7b_rep1,mistral-7b_consensus,mistral-7b_consensus_score,llama3.1-8b_rep1,llama3.1-8b_consensus,llama3.1-8b_consensus_score
0,0,A universal feature of the replication of posi...,7,7,1.0,7,7,1.0,2,2,1.0,2,2,1.0,7,7,1.0
1,1,This paper presents an ultra-low-power low-vol...,2,2,1.0,2,2,1.0,2,2,1.0,2,2,1.0,2,2,1.0
2,2,The human microbiome consists of all microorga...,8,8,1.0,8,8,1.0,7,7,1.0,3,3,1.0,7,7,1.0
3,3,Background: Limited accessibility to providers...,-1,-1,1.0,3,3,1.0,1,1,1.0,1,1,1.0,7,7,1.0
4,4,Chronic myeloid leukemia (CML) is a hematologi...,8,8,1.0,8,8,1.0,6,6,1.0,3,3,1.0,8,8,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,Objective. -To determine if bacterial vaginosi...,8,8,1.0,7,7,1.0,2,2,1.0,1,1,1.0,7,7,1.0
496,496,Substance use and depression are prevalent amo...,5,5,1.0,5,5,1.0,4,4,1.0,4,4,1.0,5,5,1.0
497,497,Aims: Understanding the metabolism of lactose ...,10,10,1.0,7,7,1.0,7,7,1.0,7,7,1.0,7,7,1.0
498,498,Physical treatments have gained great interest...,0,0,1.0,7,7,1.0,2,2,1.0,1,1,1.0,7,7,1.0


### - Metricas por modelo

In [101]:
df_annotations["ground_truth"] = ground_truth if ground_truth else None
df_metrics = annotator.evaluate_model_metrics(
    df_annotations, 
    ground_truth_col="ground_truth", 
    output_csv=True
)

12:52:36 | INFO     | Calculando métricas por modelo...
12:52:36 | INFO     | Total de linhas avaliadas: 500
12:52:36 | INFO     | Métricas para deepseek-r1-8b: Acc=0.6560, F1=0.6232, Prec=0.7387, Rec=0.6560, Cov=0.9900
12:52:36 | INFO     | Métricas para qwen3-8b: Acc=0.6240, F1=0.5842, Prec=0.7585, Rec=0.6240, Cov=0.9980
12:52:36 | INFO     | Métricas para gemma3-4b: Acc=0.5320, F1=0.5150, Prec=0.6989, Rec=0.5320, Cov=1.0000
12:52:36 | INFO     | Métricas para mistral-7b: Acc=0.4500, F1=0.4193, Prec=0.6175, Rec=0.4500, Cov=1.0000
12:52:36 | INFO     | Métricas para llama3.1-8b: Acc=0.5780, F1=0.5411, Prec=0.5790, Rec=0.5780, Cov=1.0000
12:52:36 | SUCCESS  | Métricas por modelo salvas em: ..\..\results\wos5736\model_metrics.csv
12:52:36 | SUCCESS  | ✓ Métricas calculadas com sucesso


## 5) Calcular Consenso

In [102]:
from src.llm_annotation_system.consensus.consensus_calculator import ConsensusCalculator
from src.llm_annotation_system.consensus.consensus_evaluator import ConsensusEvaluator
from src.experiments.base_experiment import EXPERIMENT_CONFIG


# Inicializar calculador
calculator = ConsensusCalculator(
    consensus_threshold=EXPERIMENT_CONFIG['consensus'].get('threshold', 0.8),
    default_strategy=EXPERIMENT_CONFIG['consensus'].get('strategy', "majority_vote")
)

analyzer = ConsensusEvaluator(categories=categories, calculator=calculator, output_dir=annotator.results_dir)

df_with_consensus = analyzer.compute_consensus(df_annotations)

12:52:36 | INFO     | Executando cálculo de consenso interno...
12:52:36 | INFO     | Calculando consenso...
12:52:36 | SUCCESS  | Consenso calculado:
12:52:36 | INFO     |   Alto (≥80%): 265 (53.0%)
12:52:36 | INFO     |   Médio (60-80%): 135 (27.0%)
12:52:36 | INFO     |   Baixo (<60%): 100 (20.0%)
12:52:36 | INFO     |   Problemáticos: 61 (12.2%)
12:52:36 | INFO     |   Itens que precisam de revisão: 0 (0.0%)
12:52:36 | SUCCESS  | Cálculo de consenso finalizado.


### - Estatisticas

In [103]:
logger.info("\n📊 Estatísticas de Consenso:")
logger.info(f"  Média: {df_with_consensus['consensus_score'].mean():.2%}")
logger.info(f"  Mediana: {df_with_consensus['consensus_score'].median():.2%}")
logger.info(f"  Desvio padrão: {df_with_consensus['consensus_score'].std():.2%}")

12:52:36 | INFO     | 
📊 Estatísticas de Consenso:
12:52:36 | INFO     |   Média: 72.36%
12:52:36 | INFO     |   Mediana: 80.00%
12:52:36 | INFO     |   Desvio padrão: 22.04%


### - Distribuição por nível

In [104]:
# Distribuição por nível
levels = df_with_consensus['consensus_level'].value_counts()
logger.info("Distribuição por nível:")
for level, count in levels.items():
    logger.info(f"  {level}: {count} ({count/len(df_with_consensus):.1%})")

12:52:36 | INFO     | Distribuição por nível:
12:52:36 | INFO     |   high: 265 (53.0%)
12:52:36 | INFO     |   medium: 135 (27.0%)
12:52:36 | INFO     |   low: 100 (20.0%)


## - Visualizar consenso

In [105]:
from src.llm_annotation_system.consensus.consensus_visualizer import ConsensusVisualizer

visualizer = ConsensusVisualizer(output_dir=annotator.results_dir)

In [106]:
visualizer.plot_score_and_levels(
    df_with_consensus=df_with_consensus,
    levels=levels
)

✓ Gráfico salvo: score_and_levels.html


## 7️⃣ Análise Detalhada de Consenso

### - Gerando Report

In [107]:
# Gerar relatório
report = analyzer.generate_consensus_report(
    df=df_with_consensus
)

logger.success("✓ Relatório gerado")

12:52:36 | INFO     | Gerando relatório completo de consenso...
12:52:36 | INFO     | Fleiss' Kappa: 0.488 (Moderado)
12:52:36 | INFO     | Casos problemáticos: 100
12:52:36 | SUCCESS  | Relatório de consenso gerado com sucesso.
12:52:36 | SUCCESS  | ✓ Relatório gerado


### - Pairwise_agreement

In [108]:
logger.info("\n📊 Gerando heatmap de concordância...")
visualizer.plot_agreement_heatmap(
    agreement_df=report['pairwise_agreement'],
    title='Matriz de Concordância entre Modelos',
)

12:52:36 | INFO     | 
📊 Gerando heatmap de concordância...


✓ Heatmap salvo: agreement_heatmap.html


### - Cohens Kappa

In [109]:
logger.info("\n📊 Gerando heatmap de Cohens_Kappa...")
visualizer.plot_kappa_heatmap(
    kappa_df=report['cohens_kappa']
)

12:52:36 | INFO     | 
📊 Gerando heatmap de Cohens_Kappa...


✓ Heatmap salvo: kappa_heatmap.html


### - Casos problemáticos

In [110]:
# Casos problemáticos
problematic = report.get('problematic_cases')
if problematic is not None and len(problematic) > 0:
    logger.warning(f"\n⚠️  {len(problematic)} casos problemáticos identificados")
    display(problematic)
else:
    logger.success("\n✓ Nenhum caso problemático identificado")

12:52:36 | WARNING  | 
⚠️  100 casos problemáticos identificados


,text_id,text,consensus_score,annotations,entropy
0,2,The human microbiome consists of all microorga...,0.4,"{8: 2, 7: 2, 3: 1}",1.521928
1,3,Background: Limited accessibility to providers...,0.4,"{-1: 1, 3: 1, 1: 2, 7: 1}",1.921928
2,8,Hosts can utilize different types of defense a...,0.4,"{8: 2, 6: 1, 3: 1, 7: 1}",1.921928
3,21,"In this paper, a new pulse width modulation (P...",0.4,"{0: 1, 1: 2, 2: 2}",1.521928
4,25,Distinct explicit and implicit memory processe...,0.4,"{6: 2, 3: 1, 2: 1, 7: 1}",1.921928
...,...,...,...,...,...
95,488,Since coal will be widely used in the next dec...,0.4,"{0: 2, 1: 2, 7: 1}",1.521928
96,489,"Calprotectin is composed of two proteins, S100...",0.4,"{10: 1, 7: 2, 1: 1, 2: 1}",1.921928
97,494,ObjectiveElectroconvulsive therapy (ECT) remai...,0.4,"{5: 1, 0: 2, 2: 1, 1: 1}",1.921928
98,495,Objective. -To determine if bacterial vaginosi...,0.4,"{8: 1, 7: 2, 2: 1, 1: 1}",1.921928


## 8️⃣ Validação com Ground Truth

In [111]:
if ground_truth:
    accuracy, cls_report, cm = analyzer.evaluate_ground_truth(
        df_with_consensus=df_with_consensus
    )

    visualizer.plot_confusion_matrix(
        cm=cm,
        categories=categories
    )
    
else:
    logger.info("⚠️ Ground truth não disponível – pulando validação")

12:52:36 | SUCCESS  | 
🎯 Accuracy: 65.80%
12:52:36 | INFO     | 
Classification Report:
{'-1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0.0}, '0': {'precision': 0.7346938775510204, 'recall': 0.8372093023255814, 'f1-score': 0.782608695652174, 'support': 43.0}, '1': {'precision': 0.5333333333333333, 'recall': 0.7272727272727273, 'f1-score': 0.6153846153846154, 'support': 33.0}, '2': {'precision': 0.8085106382978723, 'recall': 0.9047619047619048, 'f1-score': 0.8539325842696629, 'support': 42.0}, '3': {'precision': 0.6896551724137931, 'recall': 0.9302325581395349, 'f1-score': 0.7920792079207921, 'support': 43.0}, '4': {'precision': 0.972972972972973, 'recall': 0.8571428571428571, 'f1-score': 0.9113924050632911, 'support': 42.0}, '5': {'precision': 0.8666666666666667, 'recall': 0.40625, 'f1-score': 0.5531914893617021, 'support': 32.0}, '6': {'precision': 0.6363636363636364, 'recall': 0.2, 'f1-score': 0.30434782608695654, 'support': 35.0}, '7': {'precision': 0.4223602484

c:\Users\gabri\Documents\GitHub\llm-annotation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

c:\Users\gabri\Documents\GitHub\llm-annotation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

c:\Users\gabri\Documents\GitHub\llm-annotation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.



✓ Matriz de confusão salva em: ..\..\results\wos5736\graphics\confusion_matrix.html


## 9️⃣ Exportar Resultados

In [112]:
import json
from pathlib import Path

# Criar diretório
results_dir = annotator.results_dir
results_dir = results_dir.joinpath("summary")
results_dir.mkdir(parents=True, exist_ok=True)

# Salvar CSVs
df_with_consensus.to_csv(results_dir / 'dataset_anotado_completo.csv', index=False)
logger.info(f"✓ Salvos: {len(df_with_consensus)} registros")

# Alta confiança
high_conf = df_with_consensus[df_with_consensus['consensus_score'] >= 0.8]
high_conf.to_csv(results_dir / 'alta_confianca.csv', index=False)
logger.info(f"✓ Alta confiança: {len(high_conf)} registros")

# Necessita revisão
low_conf = df_with_consensus[df_with_consensus['consensus_score'] < 0.8]
low_conf.to_csv(results_dir / 'necessita_revisao.csv', index=False)
logger.info(f"✓ Necessita revisão: {len(low_conf)} registros")

# Sumário JSON
summary = {
    'dataset': {
        'name': dataset_name,
        'total_texts': len(texts),
        'categories': categories,
        'has_ground_truth': ground_truth is not None
    },
    'config': {
        'models': DEFAULT_MODELS,
        'total_models': len(annotator.models),
        'use_alternative_params': annotator.use_alternative_params,
        'num_repetitions': num_repetitions,
        'total_annotations': len(texts) * len(annotator.models) * num_repetitions
    },
    'results': {
        'consensus_mean': float(df_with_consensus['consensus_score'].mean()),
        'consensus_median': float(df_with_consensus['consensus_score'].median()),
        'high_consensus': int((df_with_consensus['consensus_level'] == 'high').sum()),
        'medium_consensus': int((df_with_consensus['consensus_level'] == 'medium').sum()),
        'low_consensus': int((df_with_consensus['consensus_level'] == 'low').sum()),
    },
    'metrics': {
        'fleiss_kappa': float(report['fleiss_kappa']),
        'fleiss_interpretation': report['fleiss_interpretation']
    }
}

if ground_truth:
    summary['validation'] = {
        'classification_report': cls_report
    }

with open(results_dir / 'sumario_experimento.json', 'w') as f:
    json.dump(summary, f, indent=2)

logger.success("\n✓ Resultados exportados com sucesso!")

12:52:36 | INFO     | ✓ Salvos: 500 registros
12:52:36 | INFO     | ✓ Alta confiança: 265 registros
12:52:36 | INFO     | ✓ Necessita revisão: 235 registros
12:52:36 | SUCCESS  | 
✓ Resultados exportados com sucesso!


## 10) Resumo Final

In [113]:
logger.info("\n" + "="*80)
logger.success("RESUMO DO EXPERIMENTO")
logger.info("="*80)

logger.info(f"\n📊 Dataset: {dataset_name}")
logger.info(f"  Textos: {len(texts)}")
logger.info(f"  Categorias: {len(categories)}")

logger.info(f"\n🤖 Configuração:")
logger.info(f"  Modelos base: {len(DEFAULT_MODELS)}")
logger.info(f"  Total modelos: {len(annotator.models)}")
logger.info(f"  Alternative params: {annotator.use_alternative_params}")
logger.info(f"  Repetições: {num_repetitions}")

logger.info(f"\n📈 Consenso:")
logger.info(f"  Média: {df_with_consensus['consensus_score'].mean():.2%}")
logger.info(f"  Fleiss' Kappa: {report['fleiss_kappa']:.3f} ({report['fleiss_interpretation']})")

if ground_truth:
    logger.info(f"\n🎯 Validação:")
    logger.info(f"  Accuracy: {accuracy:.2%}")

logger.info(f"\n📁 Arquivos gerados em: {results_dir}/")

cache_stats = annotator.get_cache_stats()
logger.info(f"\n💾 Cache: {cache_stats['total_entries']} entradas")

logger.success("\n✅ Análise completa!")

12:52:36 | INFO     | 
12:52:36 | SUCCESS  | RESUMO DO EXPERIMENTO
12:52:36 | INFO     | ================================================================================
12:52:36 | INFO     | 
📊 Dataset: wos5736
12:52:36 | INFO     |   Textos: 500
12:52:36 | INFO     |   Categorias: 11
12:52:36 | INFO     | 
🤖 Configuração:
12:52:36 | INFO     |   Modelos base: 5
12:52:36 | INFO     |   Total modelos: 5
12:52:36 | INFO     |   Alternative params: False
12:52:36 | INFO     |   Repetições: 1
12:52:36 | INFO     | 
📈 Consenso:
12:52:36 | INFO     |   Média: 72.36%
12:52:36 | INFO     |   Fleiss' Kappa: 0.488 (Moderado)
12:52:36 | INFO     | 
🎯 Validação:
12:52:36 | INFO     |   Accuracy: 65.80%
12:52:36 | INFO     | 
📁 Arquivos gerados em: ..\..\results\wos5736\summary/
12:52:36 | INFO     | 
💾 Cache: 14710 entradas
12:52:36 | SUCCESS  | 
✅ Análise completa!
